In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import awkward as ak
import uproot
import hist
import numpy as np
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea.processor import dict_accumulator, list_accumulator

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5
import itertools
import json

# for making fancy 2d histograms
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils import (plot_1d_trigger_efficiencies, 
                plot_1d_trigger_efficiencies_improvement, 
                plot_fancy_2d_trigger_efficiencies, 
                compare_trigger_efficiencies, 
                plot_cutflow,
                plot_1d_trigger_soup_compact,
                plot_1d_trigger_soup_cms)
from trig_eff_process import (create_baseline_selection_mask, 
                            TriggerEfficiencyProcessor, 
                            TriggerEfficiencyImprovementProcessor, 
                            VBFBaselineCutFlowProcessor, 
                            GeneralBaselineCutFlowProcessor,
                            TriggerSoupProcessor)

In [5]:
parking_2022 = [
        'DoubleMu4_3_Bs',
        'Dimuon14_Phi_Barrel_Seagulls',
        'Dimuon18_PsiPrime',
        'Dimuon25_Jpsi',
        'Trimuon5_3p5_2_Upsilon_Muon',
        'Mu25_TkMu0_Phi',
        'Dimuon0_Jpsi3p5_Muon2',
        'DoubleMu4_JpsiTrkTrk_Displaced',
        'Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15',
        'Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15_Charge1',
        'Dimuon18_PsiPrime_noCorrL1',
        'Dimuon24_Upsilon_noCorrL1',
        'Dimuon24_Phi_noCorrL1',
        'Dimuon25_Jpsi_noCorrL1',
        'DoubleMu5_Upsilon_DoubleEle3_CaloIdL_TrackIdL',
        'DoubleMu3_DoubleEle7p5_CaloIdL_TrackIdL_Upsilon',
        'TrimuonOpen_5_3p5_2_Upsilon_Muon',
        'DoubleMu3_TkMu_DsTau3Mu',
        'DoubleMu2_Jpsi_DoubleTrk1_Phi1p05',
        'Dimuon12_Upsilon_y1p4',
        'Mu30_TkMu0_Psi',
        'Mu30_TkMu0_Upsilon',
        'DoubleMu4_3_LowMass',
        'DoubleMu4_LowMass_Displaced',
        'DoubleMu4_MuMuTrk_Displaced',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleMu4_3_Photon4_BsToMMG',
        'DoubleMu4_3_Displaced_Photon4_BsToMMG',
        'DoubleMu4_JpsiTrk_Bc',
        'Dimuon10_Upsilon_y1p4',
        'Dimuon14_PsiPrime',
        'Dimuon14_PsiPrime_noCorrL1',
        'DoubleEle6_eta1p22_mMax6',
        'DoubleEle5p5_eta1p22_mMax6',
        'DoubleEle5_eta1p22_mMax6',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleEle6_eta1p22_mMax6',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleEle4p5_eta1p22_mMax6',
        'DoubleEle6_eta1p22_mMax6'
    ]
parking_2023 = [
        'DoubleMu4_3_Bs',
        'Dimuon14_Phi_Barrel_Seagulls',
        'Dimuon18_PsiPrime',
        'Dimuon25_Jpsi',
        'Trimuon5_3p5_2_Upsilon_Muon',
        'Mu25_TkMu0_Phi',
        'Dimuon0_Jpsi3p5_Muon2',
        'DoubleMu4_JpsiTrkTrk_Displaced',
        'Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15',
        'Tau3Mu_Mu7_Mu1_TkMu1_IsoTau15_Charge1',
        'Dimuon18_PsiPrime_noCorrL1',
        'Dimuon24_Upsilon_noCorrL1',
        'Dimuon24_Phi_noCorrL1',
        'Dimuon25_Jpsi_noCorrL1',
        'DoubleMu5_Upsilon_DoubleEle3_CaloIdL_TrackIdL',
        'DoubleMu3_DoubleEle7p5_CaloIdL_TrackIdL_Upsilon',
        'TrimuonOpen_5_3p5_2_Upsilon_Muon',
        'DoubleMu3_TkMu_DsTau3Mu',
        'DoubleMu2_Jpsi_DoubleTrk1_Phi1p05',
        'Dimuon12_Upsilon_y1p4',
        'Mu30_TkMu0_Psi',
        'Mu30_TkMu0_Upsilon',
        'DoubleMu4_3_LowMass',
        'DoubleMu4_LowMass_Displaced',
        'DoubleMu4_MuMuTrk_Displaced',
        'DoubleMu4_3_Photon4_BsToMMG',
        'DoubleMu4_3_Displaced_Photon4_BsToMMG',
        'DoubleMu4_JpsiTrk_Bc',
        'Dimuon10_Upsilon_y1p4',
        'Dimuon14_PsiPrime',
        'Dimuon14_PsiPrime_noCorrL1',
        'PFHT330PT30_QuadPFJet_75_60_45_40_TriplePFBTagDeepJet_4p5',
        'PFHT400_FivePFJet_100_100_60_30_30_DoublePFBTagDeepJet_4p5',
        'PFHT400_FivePFJet_120_120_60_30_30_DoublePFBTagDeepJet_4p5',
        'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70',
        'HT430_DisplacedDijet40_DisplacedTrack',
        'HT650_DisplacedDijet60_Inclusive',
        'HT430_DisplacedDijet40_Inclusive1PtrkShortSig5',
        'Mu10_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT350_PFMETNoMu60',
        'Mu8_TrkIsoVVL_DiPFJet40_DEta3p5_MJJ750_HTT300_PFMETNoMu60',
        'DiJet110_35_Mjj650_PFMET110',
        'DiJet110_35_Mjj650_PFMET120',
        'DiJet110_35_Mjj650_PFMET130',
        'TripleJet110_35_35_Mjj650_PFMET110',
        'TripleJet110_35_35_Mjj650_PFMET120',
        'TripleJet110_35_35_Mjj650_PFMET130',
        'Mu4_TrkIsoVVL_DiPFJet90_40_DEta3p5_MJJ750_HTT300_PFMETNoMu60',
        'QuadPFJet103_88_75_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
        'QuadPFJet103_88_75_15_PFBTagDeepJet_1p3_VBF2',
        'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
        'QuadPFJet105_88_76_15_PFBTagDeepJet_1p3_VBF2',
        'QuadPFJet111_90_80_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
        'QuadPFJet111_90_80_15_PFBTagDeepJet_1p3_VBF2',
        'VBF_DoubleMediumDeepTauPFTauHPS20_eta2p1',
        'DoublePFJets40_Mass500_MediumDeepTauPFTauHPS45_L2NN_MediumDeepTauPFTauHPS20_eta2p1',
        'QuadPFJet100_88_70_30_PNet1CvsAll0p5_VBF3Tight',
        'QuadPFJet105_88_75_30_PNet1CvsAll0p5_VBF3Tight',
        'QuadPFJet111_90_80_30_PNet1CvsAll0p6_VBF3Tight',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'DoubleEle10_eta1p22_mMax6',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'PFHT280_QuadPFJet30_PNet2BTagMean0p55',
        'PFHT280_QuadPFJet30_PNet2BTagMean0p60',
        'PFHT280_QuadPFJet35_PNet2BTagMean0p60',
        'HT430_DelayedJet40_SingleDelay1p25To1p75nsInclusive',
        'HT390_DisplacedDijet45_Inclusive1PtrkShortSig5',
        'HT390eta2p0_DisplacedDijet40_Inclusive1PtrkShortSig5',
        'VBF_DiPFJet70_40_Mjj600_Detajj2p5_DiPFJet60_JetMatchingQuadJet',
        'VBF_DiPFJet70_40_Mjj600_Detajj2p5_DiPFJet60_JetMatchingFiveJet',
        'VBF_DiPFJet70_40_Mjj600_Detajj2p5_DiPFJet60_JetMatchingSixJet',
        'VBF_DiPFJet75_40_Mjj500_Detajj2p5_PFMET85',
        'VBF_DiPFJet75_40_Mjj500_Detajj2p5_PFMET85_TriplePFJet',
        'VBF_DiPFJet90_40_Mjj600_Detajj2p5_Mu3_TrkIsoVVL',
        'VBF_DiPFJet90_40_Mjj600_Detajj2p5_Mu3_TrkIsoVVL_TriplePFJet',
        'VBF_DiPFJet45_Mjj500_Detajj2p5_MediumDeepTauPFTauHPS45_L2NN_eta2p1',
        'DoubleEle6p5_eta1p22_mMax6',
        'DoubleEle7_eta1p22_mMax6',
        'DoubleEle7p5_eta1p22_mMax6',
        'DoubleEle8_eta1p22_mMax6',
        'DoubleEle8p5_eta1p22_mMax6',
        'DoubleEle9_eta1p22_mMax6',
        'DoubleEle9p5_eta1p22_mMax6',
        'VBF_DiPFJet105_40_Mjj1000_Detajj3p5',
        'VBF_DiPFJet105_40_Mjj1000_Detajj3p5_TriplePFJet',
        'DoubleEle6_eta1p22_mMax6',
        'DoubleEle5p5_eta1p22_mMax6',
        'PFHT400_SixPFJet32_DoublePFBTagDeepJet_2p94',
        'PFHT450_SixPFJet36_PFBTagDeepJet_1p59',
        'DoubleEle5_eta1p22_mMax6',
        'DoubleEle6_eta1p22_mMax6',
        'DoubleEle5p5_eta1p22_mMax6',
        'DoubleEle6_eta1p22_mMax6',
        'DoubleEle5_eta1p22_mMax6',
        'DoubleEle5p5_eta1p22_mMax6',
        'VBF_DiPFJet45_Mjj500_Detajj2p5_Photon12',
        'VBF_DiPFJet45_Mjj500_Detajj2p5_Ele12_eta2p1_WPTight_Gsf',
        'DoubleEle5_eta1p22_mMax6',
        'PFHT400_SixPFJet32_PNet2BTagMean0p50',
        'PFHT450_SixPFJet36_PNetBTag0p35',
        'HT350_DelayedJet40_SingleDelay1p75To3p5nsInclusive',
        'VBF_DiPFJet110_40_Mjj1000_Detajj3p5',
        'VBF_DiPFJet110_40_Mjj1000_Detajj3p5_TriplePFJet',
        'VBF_DiPFJet125_45_Mjj1000_Detajj3p5',
        'VBF_DiPFJet125_45_Mjj1000_Detajj3p5_TriplePFJet',
        'VBF_DiPFJet75_45_Mjj600_Detajj2p5_DiPFJet60_JetMatchingQuadJet',
        'VBF_DiPFJet75_45_Mjj600_Detajj2p5_DiPFJet60_JetMatchingFiveJet',
        'VBF_DiPFJet75_45_Mjj600_Detajj2p5_DiPFJet60_JetMatchingSixJet',
        'VBF_DiPFJet80_45_Mjj500_Detajj2p5_PFMET85',
        'VBF_DiPFJet80_45_Mjj500_Detajj2p5_PFMET85_TriplePFJet',
        'VBF_DiPFJet95_45_Mjj600_Detajj2p5_Mu3_TrkIsoVVL',
        'VBF_DiPFJet95_45_Mjj600_Detajj2p5_Mu3_TrkIsoVVL_TriplePFJet',
        'VBF_DiPFJet45_Mjj500_Detajj2p5_Photon17',
        'VBF_DiPFJet50_Mjj500_Detajj2p5_Photon22',
        'VBF_DiPFJet45_Mjj500_Detajj2p5_Ele17_eta2p1_WPTight_Gsf',
        'VBF_DiPFJet50_Mjj500_Detajj2p5_Ele22_eta2p1_WPTight_Gsf',
        'L1Tau_DelayedJet40_SingleDelay2p75To4nsInclusive',
        'DoubleEle4p5_eta1p22_mMax6',
        'DoubleEle4p5_eta1p22_mMax6',
        'DoubleEle4p5_eta1p22_mMax6',
        'DoubleEle4_eta1p22_mMax6',
        'HT430_DelayedJet40_SingleDelay1To1p5nsInclusive',
        'HT430_DelayedJet40_SingleDelay1p1To1p6nsInclusive',
        'DoubleEle4_eta1p22_mMax6'
    ]

In [6]:
dir = '2022'
print(f'Started processing {dir}')
with open(f'/srv/{dir}/{dir}_VBF.json', 'r') as file:
    data = json.load(file)
samples = data['VBF_Hto2B']
sample = samples[0]
events = NanoEventsFactory.from_root(sample, schemaclass=NanoAODSchema).events()
for trig in parking_2022:
    if trig not in events.HLT.fields:
        print(trig)
print(f'Finished processing {dir}')

dir = '2023'
print(f'Started processing {dir}')
with open(f'/srv/{dir}/{dir}_VBF.json', 'r') as file:
    data = json.load(file)
samples = data['VBF_Hto2B']
sample = samples[0]
events = NanoEventsFactory.from_root(sample, schemaclass=NanoAODSchema).events()
for trig in parking_2023:
    if trig not in events.HLT.fields:
        print(trig)
print(f'Finished processing {dir}')



Started processing 2022
Finished processing 2022
Started processing 2023
Finished processing 2023


In [4]:
# 2022
init_soup = ['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
triggers_to_add = ['QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1', 'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65']
# 2022EE
init_soup = ['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
triggers_to_add = ['QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1', 'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65']
# 2023
# 'AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet275_SoftDropMass40_PNetBB0p10', 
init_soup = ['AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet230_SoftDropMass40']
# 'PFHT280_QuadPFJet30_PNet2BTagMean0p55', 'PFHT280_QuadPFJet30_PNet2BTagMean0p60', 'PFHT280_QuadPFJet35_PNet2BTagMean0p60', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70',
triggers_to_add = [ 'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']
# 2023BPix
# 'AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet275_SoftDropMass40_PNetBB0p10', 
init_soup = ['AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet230_SoftDropMass40']
# 'PFHT280_QuadPFJet30_PNet2BTagMean0p55', 'PFHT280_QuadPFJet30_PNet2BTagMean0p60', 'PFHT280_QuadPFJet35_PNet2BTagMean0p60', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70',
triggers_to_add = [ 'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']


In [6]:
trigger_dict = {
    '2022': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1',
            'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65'
        ]
    },
    '2022EE': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1',
            'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65'
        ]
    },
    '2023': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PNetBB0p06',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
            'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70'
        ]
    },
    '2023BPix': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PNetBB0p06',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
            'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70'
        ]
    }
}

print(trigger_dict['2022EE']['initial_soup'])
print(trigger_dict['2023']['triggers_to_add'])


['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
['QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']
